In [11]:
# Import required libraries
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

# Batch size
batch_size = 100

# Dataset transformation
transform = transforms.ToTensor()

# Download and load MNIST dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transform, download=False)

# Data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [12]:
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, latent_dim):
        super(VAE, self).__init__()

        # Encoder layers
        self.encoder_fc1 = nn.Linear(input_dim, hidden_dim1)
        self.encoder_fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc_mu = nn.Linear(hidden_dim2, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim2, latent_dim)
        
        # Decoder layers
        self.decoder_fc1 = nn.Linear(latent_dim, hidden_dim2)
        self.decoder_fc2 = nn.Linear(hidden_dim2, hidden_dim1)
        self.decoder_fc3 = nn.Linear(hidden_dim1, input_dim)
        
    def encode(self, x):
        h1 = F.relu(self.encoder_fc1(x))
        h2 = F.relu(self.encoder_fc2(h1))
        return self.fc_mu(h2), self.fc_logvar(h2)
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu
    
    def decode(self, z):
        h3 = F.relu(self.decoder_fc1(z))
        h4 = F.relu(self.decoder_fc2(h3))
        return torch.sigmoid(self.decoder_fc3(h4))
    
    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# Instantiate and check for CUDA
vae = VAE(input_dim=784, hidden_dim1=512, hidden_dim2=256, latent_dim=2)
if torch.cuda.is_available():
    vae.cuda()


In [13]:
vae

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc31): Linear(in_features=256, out_features=2, bias=True)
  (fc32): Linear(in_features=256, out_features=2, bias=True)
  (fc4): Linear(in_features=2, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [14]:
# Optimizer
optimizer = optim.Adam(vae.parameters())

# Loss function: reconstruction error + KL divergence
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD


In [15]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        if torch.cuda.is_available():
            data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, logvar = vae(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item() / len(data):.6f}')
    print(f'====> Epoch: {epoch} Average loss: {train_loss / len(train_loader.dataset):.4f}')


In [16]:
def test():
    vae.eval()
    test_loss = 0
    with torch.no_grad():
        for data, _ in test_loader:
            if torch.cuda.is_available():
                data = data.cuda()
            recon, mu, logvar = vae(data)
            test_loss += loss_function(recon, data, mu, logvar).item()
        
    test_loss /= len(test_loader.dataset)
    print(f'====> Test set loss: {test_loss:.4f}')


In [17]:
# Training and testing loop
for epoch in range(1, 51):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 543.505273
Train Epoch: 1 [10000/60000 (17%)]	Loss: 189.396211
Train Epoch: 1 [20000/60000 (33%)]	Loss: 177.549082
Train Epoch: 1 [30000/60000 (50%)]	Loss: 170.036738
Train Epoch: 1 [40000/60000 (67%)]	Loss: 160.531416
Train Epoch: 1 [50000/60000 (83%)]	Loss: 167.588437
====> Epoch: 1 Average loss: 177.2862
====> Test set loss: 161.2505
Train Epoch: 2 [0/60000 (0%)]	Loss: 158.884492
Train Epoch: 2 [10000/60000 (17%)]	Loss: 161.030801
Train Epoch: 2 [20000/60000 (33%)]	Loss: 167.027832
Train Epoch: 2 [30000/60000 (50%)]	Loss: 151.838857
Train Epoch: 2 [40000/60000 (67%)]	Loss: 154.238750
Train Epoch: 2 [50000/60000 (83%)]	Loss: 155.915732
====> Epoch: 2 Average loss: 157.1256
====> Test set loss: 154.0658
Train Epoch: 3 [0/60000 (0%)]	Loss: 155.922598
Train Epoch: 3 [10000/60000 (17%)]	Loss: 143.489014
Train Epoch: 3 [20000/60000 (33%)]	Loss: 152.752100
Train Epoch: 3 [30000/60000 (50%)]	Loss: 144.876035
Train Epoch: 3 [40000/60000 (67%)]	Loss: 158.97

In [10]:
with torch.no_grad():
    z = torch.randn(64, 2).cuda() if torch.cuda.is_available() else torch.randn(64, 2)
    sample = vae.decode(z)
    
    save_image(sample.view(64, 1, 28, 28), './samples/sample.png')

AssertionError: Torch not compiled with CUDA enabled